In [1]:
import unittest
import pandas as pd
import os, sys, io
from io import StringIO
import pkg_resources
import importlib
from colorama import Fore, Back, Style 
import ast

<h1>Link Code to be Tested</h1>

In [2]:
# Reloads the devanalyst modules used in this notebook. Useful to refresh after code changes in the devanalyst modules
def reload():
    global timecard
    timecard = importlib.import_module('devanalyst.simulation.GenerateTimecards')
    timecard = importlib.reload(timecard)

In [5]:
# Re-run this each time the source code changes, so that we have the latest version
reload()

importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\GenerateTimecards.ipynb


In [6]:
import devanalyst.simulation.GenerateTimecards as timecard
from devanalyst.simulation.GenerateTimecards import UserStoriesRepo, UserStory, UserStoryStatus, \
Ticket, TicketsRepo, ScrumTeam, ScrumTeamsRepo, IdCounter

<h1>Common Context Across Tests</h1>

<h2>Immutable Globals</h2>

In [7]:
#globals used in this test. After they are initialized, they don't change.
PM_DF = None
DEV_DF = None
TEAMS_DF = None
STORIES_REPO = timecard.UserStoriesRepo([])
TEAMS_REPO = timecard.ScrumTeamsRepo([])
TICKETS_REPO = TicketsRepo([])

<h2>Mutable Globals</h2>

In [8]:
# Global variable used to have a single counter for user stories id as they get generated in multiple calls. 
# Mutable state
NEXT_USER_STORY_ID = None

# Dictioaries of test results. Each key is the string name of a test and the value is the output of that test,
# expected or actual
EXPECTED = {}
ACTUAL = {}

<h2>Test Utilities</h2>

In [9]:
def initTestData(developers_df, productManagers_df, releaseDuration, sprintDuration=10):
# Test the code for creating user stories by constructing them and arranging them in a dataframe to display and evidence
# visually that the code works as it should

    global STORIES_REPO
    global TEAMS_REPO
    global NEXT_USER_STORY_ID
    STORIES_REPO = UserStoriesRepo([])
    TEAMS_REPO = ScrumTeamsRepo([])
    TICKETS_REPO = TicketsRepo([])
    NEXT_USER_STORY_ID = IdCounter()
    #cols = ['User Story Id','Scrum Team', 'Product Manager', 'Developer', 'Estimate',]
    userStoryId_vals = []
    scrumTeam_vals = []
    developer_vals = []
    productManager_vals = []
    estimate_vals = []
    
    teams_df = timecard.createTeamsDF(developers_df, productManagers_df)
    
    for team in teams_df['Scrum Team']:
        stories, backlog = timecard.createUserStoryBacklog(team, releaseDuration, sprintDuration, NEXT_USER_STORY_ID)
        STORIES_REPO.stories.extend(stories)
        TEAMS_REPO.teams.append(team)
        for story in stories:
            scrumTeam_vals.append(team.teamId)
            userStoryId_vals.append(story.userStoryId)
            developer_vals.append(story.developer)
            productManager_vals.append(story.productManager)
            estimate_vals.append(story.originalEstimate)
        team.backlog = backlog
    stories_dict = {'User Story Id': userStoryId_vals, 'Scrum Team': scrumTeam_vals, 'Product Manager':productManager_vals, \
                'Developer': developer_vals, 'Estimate': estimate_vals}
    return teams_df, pd.DataFrame(stories_dict)

In [10]:
def loadTestResources():
    global PM_DF
    global DEV_DF
    resource_path = '/'.join(('Resources', 'Simulation', 'Team.xlsx'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    PM_DF = pd.read_excel(fullpath, 'PMs')
    DEV_DF = pd.read_excel(fullpath, 'Dev')

loadTestResources()

In [11]:
def createExpectedOutput(expected_df, testName):
    resource_path = '/'.join(('Resources', 'Tests','Simulation', testName + '_EXPECTED.csv'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    return expected_df.to_csv(fullpath)

def loadExpectedOutput(testName, literal_cols=[]):
    resource_path = '/'.join(('Resources', 'Tests','Simulation', testName + '_EXPECTED.csv'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    df = pd.read_csv(fullpath)
    
    spurious_col = 'Unnamed: 0'
    if spurious_col in df.columns:
        df = df.drop([spurious_col], axis = 'columns') # Remove spurious index column, if any
    
    # Now for each column in list_cols, we have to treat it like it is a list so replace the contents of that
    # column with a parsing of the string value to to a list value
    for col in literal_cols:
        df[col] = df[col].apply(lambda x: ast.literal_eval(str(x)))
    return df

In [12]:
def matches(actual, expected):
    if isinstance(actual, str):
        return actual==expected
    else:
        return actual.equals(expected)

In [13]:
def testOK(testname):
    return matches(ACTUAL[testname], EXPECTED[testname])

<h1>test_uss</h1>
<p>Test User Story Status</p>

In [12]:
# Implement test logic, and run

# Helper methods to the test
def format_ticket(ticket):
    return ('Ticket:' + ticket.ticketId \
            + ',\n\t\t storyId=' + ticket.userStoryId \
            + ',\n\t\t costToFix=' + str(ticket.costToFix) \
            + ',\n\t\t effortToDate=' + str(ticket.effortToDate) \
            + ',\n\t\t percentAchieved=' + str(ticket.percentAchieved))
                
def format_tickets(tickets):
    output = ''
    for ticket in tickets:
        output = output + '\n\t\t{' + format_ticket(ticket) + '}'
    return output
            
def format_uss(uss):
    return ('\n *** USS:' + uss.userStoryId \
            + '\n\t achieved=' + str(uss.percentAchieved) \
            + ',\n\t planned=' + str(uss.planned) \
            + ',\n\t sprintPlanned=' + str(uss.sprintPlanned) \
            + ',\n\t tickets=' + format_tickets(uss.pendingTickets))

def format_item(item, item_label, sprint, timeInSprint):
    return ('\n *** ' + item_label + ' at ' + timeInSprint + ' of sprint' + str(sprint) + ': ' \
            + '\n\t userStoryId=' + str(item.userStoryId) \
            + ',\n\t taskType=' + str(item.taskType) \
            + ',\n\t ticketId=' + str(item.ticketId) \
            + ',\n\t estimate=' + '{0:.2f}'.format(item.estimate) \
            + ',\n\t percentAchieved=' + str(item.percentAchieved)  \
            + ',\n\t sprintPlanned=' + str(item.sprintPlanned))

# Test logic
def test_uss():
    output = '' 
    repo = UserStoriesRepo([UserStory('Story A', 25, 'Joe Developer', 'Amy PM'), \
                            UserStory('Story B', 17, 'Alex Developer', 'Kate PM')])
    uss = UserStoryStatus('Story B', 0.0)
    uss.planned = True
    uss.sprintPlanned = 1
    output = output + (format_uss(uss))
    item = uss.generateWorkItems(repo)[0]
    output = output + (format_item(item, 'Item#1', 1, 'start'))
    item.percentAchieved = 0.7
    newTickets = [Ticket('Bug 100','Story B', 4), Ticket('Bug 101','Story B', 1.5)]
    bugRepo = TicketsRepo(newTickets)
    output = output + (format_item(item, 'Item#1', 1, 'end'))
    uss.updateStatus([item], newTickets, bugRepo)
    uss.sprintPlanned = 2
    output = output + (format_uss(uss))

    items = uss.generateWorkItems(repo)
    item=items[0]
    output = output + (format_item(item, 'Item#1', 2, 'start'))
    item=items[1]
    output = output + (format_item(item, 'Item#2', 2, 'start'))
    item=items[2]
    output = output + (format_item(item, 'Item#3', 2, 'start'))
    items[0].percentAchieved = 0.9
    items[1].percentAchieved = 1.0
    items[2].percentAchieved = 0.5
    item=items[0]
    item=items[0]
    item=items[1]
    output = output + (format_item(item, 'Item#2', 2, 'end'))
    item=items[2]
    output = output + (format_item(item, 'Item#3', 2, 'end'))
    uss.updateStatus(items, [], bugRepo)
    output = output + (format_uss(uss))
    return output

# Run the test
test_uss_ACTUAL = test_uss()

In [13]:
# Uncomment to print a string output one can copy and paste into test_uss_EXPECTED
#test_uss_ACTUAL

In [14]:
# Set expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_uss_EXPECTED = '\n *** USS:Story B\n\t achieved=0.0,\n\t planned=True,\n\t sprintPlanned=1,\n\t tickets=\n *** Item#1 at start of sprint1: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=17.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=1\n *** Item#1 at end of sprint1: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=17.00,\n\t percentAchieved=0.7,\n\t sprintPlanned=1\n *** USS:Story B\n\t achieved=0.7,\n\t planned=True,\n\t sprintPlanned=2,\n\t tickets=\n\t\t{Ticket:Bug 100,\n\t\t storyId=Story B,\n\t\t costToFix=4,\n\t\t effortToDate=0.0,\n\t\t percentAchieved=0.0}\n\t\t{Ticket:Bug 101,\n\t\t storyId=Story B,\n\t\t costToFix=1.5,\n\t\t effortToDate=0.0,\n\t\t percentAchieved=0.0}\n *** Item#1 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=5.10,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#2 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 100,\n\t estimate=2.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#3 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 101,\n\t estimate=2.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#2 at end of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 100,\n\t estimate=2.00,\n\t percentAchieved=1.0,\n\t sprintPlanned=2\n *** Item#3 at end of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 101,\n\t estimate=2.00,\n\t percentAchieved=0.5,\n\t sprintPlanned=2\n *** USS:Story B\n\t achieved=0.97,\n\t planned=True,\n\t sprintPlanned=2,\n\t tickets=\n\t\t{Ticket:Bug 101,\n\t\t storyId=Story B,\n\t\t costToFix=1.5,\n\t\t effortToDate=2.0,\n\t\t percentAchieved=0.5}'

EXPECTED['uss'] = test_uss_EXPECTED
ACTUAL['uss'] = test_uss_ACTUAL
testOK('uss')

True

In [15]:
# Print ACTUAL output
print(Back.BLUE + Fore.WHITE + '--------------------- ACTUAL -------------------------', \
      Back.RESET + Fore.BLUE + '\n' + test_uss_ACTUAL) 

--------------------- ACTUAL ------------------------- 

 *** USS:Story B
	 achieved=0.0,
	 planned=True,
	 sprintPlanned=1,
	 tickets=
 *** Item#1 at start of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.0,
	 sprintPlanned=1
 *** Item#1 at end of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.7,
	 sprintPlanned=1
 *** USS:Story B
	 achieved=0.7,
	 planned=True,
	 sprintPlanned=2,
	 tickets=
		{Ticket:Bug 100,
		 storyId=Story B,
		 costToFix=4,
		 effortToDate=0.0,
		 percentAchieved=0.0}
		{Ticket:Bug 101,
		 storyId=Story B,
		 costToFix=1.5,
		 effortToDate=0.0,
		 percentAchieved=0.0}
 *** Item#1 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=5.10,
	 percentAchieved=0.0,
	 sprintPlanned=2
 *** Item#2 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=BUGS_ON_UNFINISHED_STORIES

In [16]:
# Print EXPECTED output
print(Back.GREEN + Fore.WHITE + '--------------------- EXPECTED -----------------------', \
      Back.RESET + Fore.GREEN + '\n' + test_uss_EXPECTED) 

--------------------- EXPECTED ----------------------- 

 *** USS:Story B
	 achieved=0.0,
	 planned=True,
	 sprintPlanned=1,
	 tickets=
 *** Item#1 at start of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.0,
	 sprintPlanned=1
 *** Item#1 at end of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.7,
	 sprintPlanned=1
 *** USS:Story B
	 achieved=0.7,
	 planned=True,
	 sprintPlanned=2,
	 tickets=
		{Ticket:Bug 100,
		 storyId=Story B,
		 costToFix=4,
		 effortToDate=0.0,
		 percentAchieved=0.0}
		{Ticket:Bug 101,
		 storyId=Story B,
		 costToFix=1.5,
		 effortToDate=0.0,
		 percentAchieved=0.0}
 *** Item#1 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=5.10,
	 percentAchieved=0.0,
	 sprintPlanned=2
 *** Item#2 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=BUGS_ON_UNFINISHED_STORIES

<h1>test_createTeams</h1>

In [17]:
# Implement test logic, and run

TEAMS_DF = timecard.createTeamsDF(DEV_DF, PM_DF)

object_cols = ['Scrum Team'] # Need to drop these since they print always-chaning memory address + can't save it in EXPECTED
test_createTeams_ACTUAL = TEAMS_DF.drop(object_cols, axis='columns')

In [18]:
# Uncomment to update expected output to match the actual one
#createExpectedOutput(test_createTeams_ACTUAL, 'test_createTeams')

In [19]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
list_cols = ['Developers', 'Product Managers', 'Areas of Responsibility']
test_createTeams_EXPECTED = loadExpectedOutput('test_createTeams', list_cols)

EXPECTED['createTeams'] = test_createTeams_EXPECTED
ACTUAL['createTeams'] = test_createTeams_ACTUAL
testOK('createTeams')

True

In [20]:
test_createTeams_ACTUAL

,Team Id,Developers,Product Managers,Areas of Responsibility
0,Team A,"[Anton Easterday, Beau Hockensmith, Bruno Stud...","[Sherlyn Cordle, Edgar Hibbler]","[Doctor, Patient]"
1,Team B,"[Hyun Jaffe, Isaura Casterline, Jacinto Immel,...",[Spencer Venezia],[Ministry of Health]
2,Team C,"[Lonnie Belz, Lorriane Demmer, Margorie Bering...",[Jamie Addington],[Hospital Administration]
3,Team D,"[Mohammad Tineo, Nohemi Santini, Olevia Haymak...",[Georgine Roan],[Insurance]


In [21]:
test_createTeams_EXPECTED

,Team Id,Developers,Product Managers,Areas of Responsibility
0,Team A,"[Anton Easterday, Beau Hockensmith, Bruno Stud...","[Sherlyn Cordle, Edgar Hibbler]","[Doctor, Patient]"
1,Team B,"[Hyun Jaffe, Isaura Casterline, Jacinto Immel,...",[Spencer Venezia],[Ministry of Health]
2,Team C,"[Lonnie Belz, Lorriane Demmer, Margorie Bering...",[Jamie Addington],[Hospital Administration]
3,Team D,"[Mohammad Tineo, Nohemi Santini, Olevia Haymak...",[Georgine Roan],[Insurance]


<h1>test_userStoryCreate</h1>
<p>This test has multiple views, each of which is checked separately</p>
<li>test_userStoryCreate_stories
<li>test_userStoryCreate_estimates
<li>test_userStoryCreate_crossCheck
<li>test_userStoryCreate_workload

In [250]:
# Implement test logic, and run it

#Test logic
def test_userStoryCreate():
    output = {}
    
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    
    teams_df, stories_df = initTestData(DEV_DF, PM_DF, 125)
    
    grouped_estimates_df = stories_df.groupby([ 'Scrum Team', 'Developer'])['Estimate'].sum()
    workload_df = stories_df.groupby([ 'Scrum Team'])['User Story Id'].count()
    #workload_count_df = workload_df.count()
    
    avg_estimates_df = grouped_estimates_df.unstack().apply(lambda x: x.mean(), axis='columns')

    # Reset index to match the way how EXPECTED will be saved as a CSV file
    estimates_df      = grouped_estimates_df.reset_index()
    workload_df       = workload_df.reset_index()
    avg_estimates_df= avg_estimates_df.reset_index()
    
    # The unstacking above created an column with a number as the column name, 0. That will not match once expected output is saved
    # and reloaded, as it will come back as the string '0'. So rename that column to avoid spurious test failures
    avg_estimates_df = avg_estimates_df.rename(index=str, columns={0: 'Avg'})
    
    # Because of the manipulations, the index has changed and that will cause mistaches with the EXPECTED loaded from 
    # CSV. So re-index
    avg_estimates_df.index = pd.RangeIndex(start=0, stop=avg_estimates_df.index.size, step=1)
    
    crosscheck = [len(teams_df['Scrum Team'][0].backlog.pendingUserStories), 
                  len(teams_df['Scrum Team'][1].backlog.pendingUserStories),
                  len(teams_df['Scrum Team'][2].backlog.pendingUserStories), 
                  len(teams_df['Scrum Team'][3].backlog.pendingUserStories)]
    crosscheck_df = pd.DataFrame({'Team idx': [0,1,2,3], 'Backlog size': crosscheck})
    
    output['stories_df'] = stories_df
    output['estimates_df'] = estimates_df
    output['workload_df'] = workload_df
    output['crosscheck_df'] = crosscheck_df
    output['avg_estimates_df'] = avg_estimates_df

    return output

# Run the test
test_userStoryCreate_ACTUAL = test_userStoryCreate()

In [251]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_userStoryCreate_EXPECTED():
    createExpectedOutput(test_userStoryCreate_ACTUAL['stories_df'],        'test_userStoryCreate.stories_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['estimates_df'],      'test_userStoryCreate.estimates_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['workload_df'],       'test_userStoryCreate.workload_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['crosscheck_df'],     'test_userStoryCreate.crosscheck_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['avg_estimates_df'],  'test_userStoryCreate.avg_estimates_df')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_userStoryCreate_EXPECTED()

In [252]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK

test_userStoryCreate_EXPECTED = {}

test_userStoryCreate_EXPECTED['stories_df']         = loadExpectedOutput('test_userStoryCreate.stories_df')
test_userStoryCreate_EXPECTED['estimates_df']       = loadExpectedOutput('test_userStoryCreate.estimates_df')
test_userStoryCreate_EXPECTED['workload_df']        = loadExpectedOutput('test_userStoryCreate.workload_df')
test_userStoryCreate_EXPECTED['crosscheck_df']      = loadExpectedOutput('test_userStoryCreate.crosscheck_df')
test_userStoryCreate_EXPECTED['avg_estimates_df']   = loadExpectedOutput('test_userStoryCreate.avg_estimates_df')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for the sensitive fields

rounded = test_userStoryCreate_EXPECTED['avg_estimates_df']['Avg'].apply(lambda x: round(x, 6)) # Round to 6 decimal places
test_userStoryCreate_EXPECTED['avg_estimates_df']['Avg'] = rounded

rounded = test_userStoryCreate_ACTUAL['avg_estimates_df']['Avg'].apply(lambda x: round(x, 6)) # Round to 6 decimal places
test_userStoryCreate_ACTUAL['avg_estimates_df']['Avg'] = rounded

EXPECTED['test_userStoryCreate.stories_df']         = test_userStoryCreate_EXPECTED['stories_df']
EXPECTED['test_userStoryCreate.estimates_df']       = test_userStoryCreate_EXPECTED['estimates_df']
EXPECTED['test_userStoryCreate.workload_df']        = test_userStoryCreate_EXPECTED['workload_df']
EXPECTED['test_userStoryCreate.crosscheck_df']      = test_userStoryCreate_EXPECTED['crosscheck_df']
EXPECTED['test_userStoryCreate.avg_estimates_df']   = test_userStoryCreate_EXPECTED['avg_estimates_df']

ACTUAL['test_userStoryCreate.stories_df']           = test_userStoryCreate_ACTUAL['stories_df']
ACTUAL['test_userStoryCreate.estimates_df']         = test_userStoryCreate_ACTUAL['estimates_df']
ACTUAL['test_userStoryCreate.workload_df']          = test_userStoryCreate_ACTUAL['workload_df']
ACTUAL['test_userStoryCreate.crosscheck_df']        = test_userStoryCreate_ACTUAL['crosscheck_df']
ACTUAL['test_userStoryCreate.avg_estimates_df']     = test_userStoryCreate_ACTUAL['avg_estimates_df']

testOK('test_userStoryCreate.stories_df'), \
testOK('test_userStoryCreate.estimates_df'), \
testOK('test_userStoryCreate.workload_df'), \
testOK('test_userStoryCreate.crosscheck_df'), \
testOK('test_userStoryCreate.avg_estimates_df')

(True, True, True, True, True)

In [253]:
test_userStoryCreate_ACTUAL['stories_df']

,User Story Id,Scrum Team,Product Manager,Developer,Estimate
0,UserStory #1,Team A,Sherlyn Cordle,Beau Hockensmith,10
1,UserStory #2,Team A,Sherlyn Cordle,Glenna Mcghie,10
2,UserStory #3,Team A,Edgar Hibbler,Francisco Hoppe,4
3,UserStory #4,Team A,Sherlyn Cordle,Beau Hockensmith,7
4,UserStory #5,Team A,Edgar Hibbler,Gregorio Darr,3
5,UserStory #6,Team A,Sherlyn Cordle,Beau Hockensmith,3
6,UserStory #7,Team A,Sherlyn Cordle,Anton Easterday,8
7,UserStory #8,Team A,Edgar Hibbler,Anton Easterday,9
8,UserStory #9,Team A,Sherlyn Cordle,Gregorio Darr,9
9,UserStory #10,Team A,Sherlyn Cordle,Gregorio Darr,9


In [254]:
test_userStoryCreate_EXPECTED['stories_df']

,User Story Id,Scrum Team,Product Manager,Developer,Estimate
0,UserStory #1,Team A,Sherlyn Cordle,Beau Hockensmith,10
1,UserStory #2,Team A,Sherlyn Cordle,Glenna Mcghie,10
2,UserStory #3,Team A,Edgar Hibbler,Francisco Hoppe,4
3,UserStory #4,Team A,Sherlyn Cordle,Beau Hockensmith,7
4,UserStory #5,Team A,Edgar Hibbler,Gregorio Darr,3
5,UserStory #6,Team A,Sherlyn Cordle,Beau Hockensmith,3
6,UserStory #7,Team A,Sherlyn Cordle,Anton Easterday,8
7,UserStory #8,Team A,Edgar Hibbler,Anton Easterday,9
8,UserStory #9,Team A,Sherlyn Cordle,Gregorio Darr,9
9,UserStory #10,Team A,Sherlyn Cordle,Gregorio Darr,9


In [255]:
test_userStoryCreate_ACTUAL['estimates_df']

,Scrum Team,Developer,Estimate
0,Team A,Anton Easterday,117
1,Team A,Beau Hockensmith,125
2,Team A,Bruno Studley,122
3,Team A,Craig Garlitz,125
4,Team A,Francisco Hoppe,125
5,Team A,Glenna Mcghie,122
6,Team A,Gregorio Darr,120
7,Team A,Heriberto Martini,122
8,Team B,Hyun Jaffe,123
9,Team B,Isaura Casterline,114


In [256]:
test_userStoryCreate_EXPECTED['estimates_df']

,Scrum Team,Developer,Estimate
0,Team A,Anton Easterday,117
1,Team A,Beau Hockensmith,125
2,Team A,Bruno Studley,122
3,Team A,Craig Garlitz,125
4,Team A,Francisco Hoppe,125
5,Team A,Glenna Mcghie,122
6,Team A,Gregorio Darr,120
7,Team A,Heriberto Martini,122
8,Team B,Hyun Jaffe,123
9,Team B,Isaura Casterline,114


In [257]:
test_userStoryCreate_ACTUAL['workload_df']

,Scrum Team,User Story Id
0,Team A,185
1,Team B,125
2,Team C,144
3,Team D,205


In [258]:
test_userStoryCreate_EXPECTED['workload_df']

,Scrum Team,User Story Id
0,Team A,185
1,Team B,125
2,Team C,144
3,Team D,205


In [259]:
test_userStoryCreate_ACTUAL['crosscheck_df']

,Team idx,Backlog size
0,0,185
1,1,125
2,2,144
3,3,205


In [260]:
test_userStoryCreate_EXPECTED['crosscheck_df']

,Team idx,Backlog size
0,0,185
1,1,125
2,2,144
3,3,205


In [261]:
test_userStoryCreate_ACTUAL['avg_estimates_df']

,Scrum Team,Avg
0,Team A,122.250000
1,Team B,119.333333
2,Team C,121.571429
3,Team D,122.888889


In [262]:
test_userStoryCreate_EXPECTED['avg_estimates_df']

,Scrum Team,Avg
0,Team A,122.250000
1,Team B,119.333333
2,Team C,121.571429
3,Team D,122.888889


<h1>test_FOO: template for tests that produce a DataFrame output </h1>
<li> Copy to create a new test, and replace 'FOO' by the name of your test.
<li> Implement test logic in test_FOO()
<li> To create expected output once test logic is producing right output, temporarily uncomment the call to createExpectedOutput, call it, and comment it again. This saves expected output as a CSV file so subsequent runs of the test can load it and verify that the test still produces the same output as before.
<li> Uncomment lines that load expected output and remove the dummy line next to it creating a dummy expected output.
<li> Display the outputs for convenience in debugging when test fails.

In [ ]:
# Implement test logic, and run it

#Test logic
def test_FOO():
    return pd.DataFrame({'Dummy': [45, 34]})

# Run the test
test_FOO_ACTUAL = test_FOO()

In [ ]:
# Uncomment to update expected output to match the actual one
#createExpectedOutput(test_FOO_ACTUAL, 'test_FOO')

In [ ]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
list_cols = [] #Names of columns in expected DataFrame output that are lists

test_FOO_EXPECTED = pd.DataFrame({'Dummy': [45, 35]}) # Uncomment next line and delete this one
#test_FOO_EXPECTED = loadExpectedOutput('test_FOO', list_cols)

EXPECTED['FOO'] = test_FOO_EXPECTED
ACTUAL['FOO'] = test_FOO_ACTUAL
testOK('FOO')

In [ ]:
test_FOO_ACTUAL

In [ ]:
test_FOO_EXPECTED

In [ ]:
# Now test many sprints into the future, to see if eventually people have extra time and start using that extra time
# in the current sprint to get a head start on tasks for the next sprint
#
def testMultipleSprints(numberOfSprints, teamId, teamsRepo, ticketsRepo, storiesRepo, sprintDuration):
    work = None
    for i in range(numberOfSprints):
        work = chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, sprintDuration, sprint=i+1)
        if (i== numberOfSprints -1):
            break
        deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, sprintDuration) # This mutates 'work'
        inflow = inflowOfTickets(teamId, teamsRepo, ticketsRepo, storiesRepo)
        updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, inflow) # Does not mutate 'work'
    return work

#teams_df, stories_df = initTestData(dev_df, pm_df, 125)
#last = testMultipleSprints(11, teamId0, TEAMS_REPO, TICKETS_REPO, STORIES_REPO, 10)

<h1>Run all the tests in this notebook</h1>

In [ ]:
# Uncomment to run all the tests defined in this notebook
#unittest.main(argv=[''], verbosity=2, exit=False)